 Task - 3 :- Colorization of historical photographs.

In [ ]:
!git clone https://github.com/JoeyNMT/InstColorization.git
%cd InstColorization
!pip install -r requirements.txt


Cloning into 'InstColorization'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: 'InstColorization'
/content
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving colorization_release_v2 (3).caffemodel to colorization_release_v2 (3).caffemodel


In [ ]:
from google.colab import files
uploaded = files.upload()
img_path = list(uploaded.keys())[0]


Saving world war 2.jpg to world war 2.jpg


In [ ]:
import torch
import fastai
import matplotlib.pyplot as plt
import requests
from pathlib import Path
from PIL import Image
import warnings
import os

# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning, module="torchvision.models._utils")

# Set the working directory to DeOldify
os.chdir("/content/DeOldify")
print("✅ Current Working Directory:", os.getcwd())

# Import DeOldify Colorizer class
from deoldify.visualize import get_image_colorizer

# Load the DeOldify model
colorizer = get_image_colorizer(artistic=True, render_factor=45)

# Create the results directory if it doesn't exist
Path("./results").mkdir(exist_ok=True)

# Dictionary mapping historical periods to render factors
HISTORICAL_PERIODS = {
    "1920s": 30,
    "World War II": 40,
    "Victorian Era": 25,
    "1960s": 50
}

def get_image(image_source):
    """Loads an image from either a URL or a local file path."""
    img_path = Path("/content/grayscale.jpg")  # Temp path for processing

    if os.path.exists(image_source):  # Check if it's a local file
        img_path = Path(image_source)
        print(f"✅ Using local image: {img_path}")
    else:  # Assume it's a URL
        response = requests.get(image_source, stream=True)
        if response.status_code == 200:
            with open(img_path, "wb") as file:
                for chunk in response.iter_content(1024):
                    file.write(chunk)
            print("✅ Image downloaded from URL")
        else:
            raise ValueError("Invalid image URL or file path!")

    return img_path

def visualize_colorization_steps(image_source, period):
    """Colorizes an image (local file or URL) step by step and visualizes the process."""

    if period not in HISTORICAL_PERIODS:
        raise ValueError("Selected period not available. Choose from: " + ", ".join(HISTORICAL_PERIODS.keys()))

    render_factor = HISTORICAL_PERIODS[period]

    # Load the image (either from local file or download from URL)
    img_path = get_image(image_source)

    # Load and display the original grayscale image
    original_img = Image.open(img_path)
    plt.figure(figsize=(5,5))
    plt.imshow(original_img, cmap="gray")
    plt.axis("off")
    plt.title("Original Grayscale Image")
    plt.show()

    # Progressive colorization at different render factors
    steps = [10, 20, render_factor]  # Show intermediate colorization stages
    fig, axes = plt.subplots(1, len(steps), figsize=(15, 5))

    for i, rf in enumerate(steps):
        colorized_img_path = colorizer.plot_transformed_image(
            path=str(img_path), render_factor=rf, display_render_factor=False, results_dir=Path("./results")
        )
        colorized_img = Image.open(colorized_img_path)

        axes[i].imshow(colorized_img)
        axes[i].axis("off")
        axes[i].set_title(f"Colorized (Render Factor: {rf})")

    plt.show()
    print(f"✅ Colorization completed for the {period} period!")

# Example Usage:
# For a local image file:
visualize_colorization_steps("/content/world war 2.jpg", "1920s")

# For an online image URL:
visualize_colorization_steps("https://media.istockphoto.com/id/182812178/vector/victorian-london-dudley-street-seven-dials.jpg?s=612x612&w=0&k=20&c=fvwiPsWa-c_JEWcYQKzJ0X4aUPJzbFZmSYQ9c-4rpXU=", "Victorian Era")
